In [1]:
import numpy as np

In [2]:
# Exact RS integral diftaction expression from circular hole

import numpy as np
import matplotlib.pyplot as plt

# Constants
wavelength = 650e-9  # Wavelength of light (in meters, e.g., 650 nm)
k = 2 * np.pi / wavelength  # Wave number
R = 1e-3  # Radius of the circular aperture (in meters)
z = 1.0  # Observation distance (in meters)
num_points = 500  # Number of discretization points on the aperture

# Generate grid of points on the aperture (a circle in the xy-plane)
theta = np.linspace(0, 2 * np.pi, num_points)
x_aperture = R * np.cos(theta)
y_aperture = R * np.sin(theta)

# Observation grid (along z-axis, assume observation point is on the x-axis)
x_obs = np.linspace(-2*R, 2*R, 200)
y_obs = np.linspace(-2*R, 2*R, 200)
X_obs, Y_obs = np.meshgrid(x_obs, y_obs)

# Initialize the diffraction field at the observation points
U_obs = np.zeros(X_obs.shape, dtype=complex)

# Perform Rayleigh-Sommerfeld Integral
for i in range(num_points):
    for j in range(len(x_obs)):
        for k in range(len(y_obs)):
            # Compute distance r from each point on the aperture to the observation point
            r = np.sqrt((X_obs[j, k] - x_aperture[i])**2 + (Y_obs[j, k] - y_aperture[i])**2 + z**2)
            
            # Compute the contribution to the diffraction field
            U_obs[j, k] += np.exp(1j * k * r) / r  # Rayleigh-Sommerfeld kernel (ignoring aperture field)

# Normalize the diffraction field
U_obs /= np.max(np.abs(U_obs))

# Plot the diffraction pattern (magnitude of the field)
plt.figure(figsize=(8, 6))
plt.imshow(np.abs(U_obs)**2, extent=[x_obs.min(), x_obs.max(), y_obs.min(), y_obs.max()])
plt.colorbar(label='Intensity')
plt.title('Diffraction Pattern from a Circular Aperture')
plt.xlabel('x (m)')
plt.ylabel('y (m)')
plt.show()



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/py

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

In [3]:
# Far field approximation -- much faster

import numpy as np
import matplotlib.pyplot as plt

# Constants
wavelength = 650e-9  # Wavelength of light (in meters, e.g., 650 nm)
k = 2 * np.pi / wavelength  # Wave number
R = 1e-3  # Radius of the circular aperture (in meters)
z = 1.0  # Observation distance (in meters)
num_points = 512  # Grid size (power of 2 for efficient FFT)

# Define the size of the grid (in meters)
grid_size = 5e-3  # 5 mm grid for observation
dx = grid_size / num_points  # Grid spacing in x and y

# Generate grid for observation points (2D plane)
x_obs = np.linspace(-grid_size / 2, grid_size / 2, num_points)
y_obs = np.linspace(-grid_size / 2, grid_size / 2, num_points)
X_obs, Y_obs = np.meshgrid(x_obs, y_obs)

# Create the binary circular aperture function (1 inside the circle, 0 outside)
X_aperture = np.linspace(-R, R, num_points)
Y_aperture = np.linspace(-R, R, num_points)
X_aperture, Y_aperture = np.meshgrid(X_aperture, Y_aperture)
aperture = (X_aperture**2 + Y_aperture**2 <= R**2).astype(float)

# Compute the Fourier transform of the aperture (Far-field diffraction pattern)
# Apply FFT and shift to center the zero frequency component
aperture_ft = np.fft.fftshift(np.fft.fft2(aperture))

# Calculate the spatial frequencies corresponding to the observation grid
fx = np.fft.fftshift(np.fft.fftfreq(num_points, dx))
fy = np.fft.fftshift(np.fft.fftfreq(num_points, dx))
FX, FY = np.meshgrid(fx, fy)

# Calculate the radial frequency (in terms of wave number)
radius_freq = np.sqrt(FX**2 + FY**2)

# Use the diffraction condition: scaling by (wavelength * z)
# The scaling factor comes from the Fraunhofer approximation
scaling_factor = np.exp(1j * k * z) * (wavelength * z) / (k)

# Compute the diffraction pattern using the scaled Fourier transform
U_obs = np.abs(scaling_factor * aperture_ft)**2

# Normalize and display the diffraction pattern
U_obs /= np.max(U_obs)  # Normalize to max value for visualization

# Plot the diffraction pattern (intensity)
plt.figure(figsize=(8, 6))
plt.imshow(U_obs, extent=[x_obs.min(), x_obs.max(), y_obs.min(), y_obs.max()], cmap='hot')
plt.colorbar(label='Intensity')
plt.title('Diffraction Pattern from a Circular Aperture (FFT-based)')
plt.xlabel('x (m)')
plt.ylabel('y (m)')
plt.show()



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/py

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import